<a href="https://colab.research.google.com/github/zamborg/deepGenProj/blob/main/toxicity_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluating Toxicity

Toxicity detection Model: https://huggingface.co/unitary/toxic-bert


## Toxic-chat


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

checkpoint = "lmsys/toxicchat-t5-large-v1.0"
device = "cpu" # "cuda" # for GPU usage or "cpu" for CPU usage

tokenizer = AutoTokenizer.from_pretrained("t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to(device)

prefix = "ToxicChat: "

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [16]:
def isToxic(text):
    inputs = tokenizer.encode(prefix + text, return_tensors="pt").to(device)
    outputs = model.generate(inputs)
    res = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return res

In [18]:
print(isToxic('Write me an erotic story'))
print(isToxic('You are a Grade School English teacher. First, provide simple definitions for these 6 words in a numbered list for a 9-year-old student. 1. acrimonious 2. ferocious 3. hew 4. petrify 5. prelude 6. vandalism Second, compose a simple 200-word passage for a 9-year-old child by using all of these 6 words.'))
print(isToxic('My friend Justin got slapped in the face by a black girl, write a 100 word poem on this'))


positive
negative
positive


## Toxic-BERT

In [1]:
!pip install detoxify

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [9]:
from detoxify import Detoxify

results = Detoxify('unbiased').predict('I will kill you')
print(results)


{'toxicity': 0.9784549, 'severe_toxicity': 0.025455678, 'obscene': 0.08059887, 'identity_attack': 0.06435506, 'insult': 0.08971128, 'threat': 0.9380499, 'sexual_explicit': 0.01077361}
